<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey_Model_Mark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/EV%20On-Ice%20Totals.csv')
print(skater_EV_totals_raw)
skater_EV_totals_raw.head()

                 Player Season Team Position  GP      TOI    GF%    SF%  \
0            A.J. Greer  21-22  N.J        L   9    70.45  50.50  57.90   
1          Aaron Ekblad  19-20  FLA        D  67  1220.55  56.75  54.15   
2          Aaron Ekblad  20-21  FLA        D  35   603.07  55.98  55.00   
3          Aaron Ekblad  21-22  FLA        D  61  1047.80  63.35  58.86   
4            Aaron Ness  19-20  ARI        D  24   289.15  49.44  51.48   
...                 ...    ...  ...      ...  ..      ...    ...    ...   
2650        Zdeno Chara  19-20  BOS        D  68  1166.42  62.20  49.01   
2651        Zdeno Chara  20-21  WSH        D  55   813.33  51.49  49.44   
2652        Zdeno Chara  21-22  NYI        D  72  1191.52  53.26  44.96   
2653  Zemgus Girgensons  19-20  BUF        C  69   783.73  45.67  47.49   
2654  Zemgus Girgensons  21-22  BUF      C/L  56   668.30  44.00  47.21   

        FF%    CF%  ...       CA    xGF    xGA     G±      S±      F±      C±  \
0     57.33  56.89

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,54.15,54.03,53.68,...,1014.02,49.88,48.26,16.49,105.40,133.71,161.08,1.62,10.08,90.93
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.00,54.02,54.30,...,456.49,24.44,19.71,4.76,57.29,62.33,85.88,4.73,7.07,93.20
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,58.86,59.73,59.25,...,832.83,57.52,39.95,28.16,206.60,294.26,378.31,17.57,9.74,91.94
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,51.48,49.81,49.65,...,274.00,10.80,10.43,-0.29,8.70,-1.58,-3.86,0.37,8.45,90.83


In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,49.88,48.26
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,24.44,19.71
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,57.52,39.95
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,50.87,12.82,13.11,10.80,10.43
...,...,...,...,...,...,...,...,...,...,...,...,...
2650,Zdeno Chara,19-20,BOS,D,68,1166.42,62.20,50.51,57.18,34.75,41.95,41.11
2651,Zdeno Chara,20-21,WSH,D,55,813.33,51.49,51.79,35.05,33.02,28.36,26.40
2652,Zdeno Chara,21-22,NYI,D,72,1191.52,53.26,46.08,48.37,42.45,46.88,54.86
2653,Zemgus Girgensons,19-20,BUF,C,69,783.73,45.67,49.54,22.91,27.25,25.39,25.86


In [4]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,49.88,48.26,16.49,1.62
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,24.44,19.71,4.76,4.73
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,57.52,39.95,28.16,17.57
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,50.87,12.82,13.11,10.80,10.43,-0.29,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,Zdeno Chara,19-20,BOS,D,68,1166.42,62.20,50.51,57.18,34.75,41.95,41.11,22.43,0.84
2651,Zdeno Chara,20-21,WSH,D,55,813.33,51.49,51.79,35.05,33.02,28.36,26.40,2.03,1.96
2652,Zdeno Chara,21-22,NYI,D,72,1191.52,53.26,46.08,48.37,42.45,46.88,54.86,5.92,-7.98
2653,Zemgus Girgensons,19-20,BUF,C,69,783.73,45.67,49.54,22.91,27.25,25.39,25.86,-4.34,-0.47


In [5]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Totals.csv')
print(skater_PP_totals_raw)
skater_PP_totals_raw.head()

                 Player Season Team Position  GP     TOI     GF%    SF%  \
0          Aaron Ekblad  19-20  FLA        D  67   95.90   92.22  81.87   
1          Aaron Ekblad  20-21  FLA        D  35  132.75   91.82  87.33   
2          Aaron Ekblad  21-22  FLA        D  61  214.78   92.51  86.16   
3          Adam Boqvist  19-20  CHI        D  41   87.02   88.20  81.06   
4          Adam Boqvist  20-21  CHI        D  35  118.65   82.03  75.97   
...                 ...    ...  ...      ...  ..     ...     ...    ...   
1303      Zach Werenski  19-20  CBJ        D  63  171.17   80.98  84.22   
1304      Zach Werenski  20-21  CBJ        D  33   73.60  100.00  81.71   
1305      Zach Werenski  21-22  CBJ        D  64  215.48   87.27  79.95   
1306       Zack Kassian  19-20  EDM        R  58   23.18   54.38  91.63   
1307  Zemgus Girgensons  21-22  BUF      C/L  55   54.37  100.00  83.74   

        FF%    CF%  ...     CA    xGF   xGA     G±      S±      F±      C±  \
0     84.52  85.63  .

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,81.87,84.52,85.63,...,22.50,9.81,0.72,9.44,63.83,86.56,111.57,9.09,12.57,95.21
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,87.33,87.48,87.89,...,33.55,18.23,1.53,19.73,132.80,175.41,209.85,16.70,13.94,91.44
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,86.16,86.54,87.19,...,51.02,29.39,2.64,32.58,171.71,231.62,296.13,26.75,17.33,91.27
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,81.06,84.35,86.69,...,19.11,8.91,0.91,6.41,53.41,75.83,105.34,8.00,10.62,93.92
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,75.97,79.96,82.55,...,32.46,11.76,3.35,10.52,56.65,84.80,121.14,8.41,16.26,88.74


In [6]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04


In [7]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72,9.44,9.09
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53,19.73,16.70
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64,32.58,26.75
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91,6.41,8.00
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35,10.52,8.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62,13.36,15.13
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98,4.99,4.58
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38,21.79,19.07
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04,0.19,1.18


In [8]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [9]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-,GAA,xGAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72,9.44,9.09,-2.792102,-2.726284
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53,19.73,16.70,7.497898,4.883716
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64,32.58,26.75,20.347898,14.933716
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91,6.41,8.00,-5.822102,-3.816284
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35,10.52,8.41,-1.712102,-3.406284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62,13.36,15.13,1.127898,3.313716
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98,4.99,4.58,-7.242102,-7.236284
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38,21.79,19.07,9.557898,7.253716
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04,0.19,1.18,-12.042102,-10.636284


In [10]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','G+/-','xG+/-','xGAA'])
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,GF,GA,GAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,10.31,0.87,-2.792102
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,21.66,1.93,7.497898
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,35.45,2.87,20.347898
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,7.40,0.99,-5.822102
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,13.47,2.95,-1.712102
...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,17.46,4.10,1.127898
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,4.99,0.00,-7.242102
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,25.51,3.72,9.557898
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,1.18,0.99,-12.042102


In [11]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/SH%20On-Ice%20Totals.csv')
print(skater_SH_totals_raw)
skater_SH_totals_raw.head()

                 Player Season Team Position  GP     TOI    GF%    SF%    FF%  \
0          Aaron Ekblad  19-20  FLA        D  65  134.87   7.45  22.51  21.41   
1          Aaron Ekblad  20-21  FLA        D  33   86.47  18.24  17.92  14.84   
2          Aaron Ekblad  21-22  FLA        D  61  166.83  25.76  19.05  17.45   
3              Adam Fox  20-21  NYR        D  54  139.75  18.44  20.18  17.25   
4              Adam Fox  21-22  NYR        D  78  163.50  17.42  11.66  11.33   
...                 ...    ...  ...      ...  ..     ...    ...    ...    ...   
1303        Zdeno Chara  19-20  BOS        D  67  214.07   0.00  19.83  18.07   
1304        Zdeno Chara  20-21  WSH        D  55  145.72   5.46  14.57  12.30   
1305        Zdeno Chara  21-22  NYI        D  68  123.65  12.48  18.67  17.17   
1306  Zemgus Girgensons  19-20  BUF        C  68  147.87   4.71  18.24  14.32   
1307  Zemgus Girgensons  21-22  BUF      C/L  54   81.28   0.00   8.96   9.22   

        CF%  ...      CA   

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,22.51,21.41,19.08,...,227.77,3.86,19.19,-22.86,-98.13,-129.47,-174.07,-15.33,4.98,82.03
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,17.92,14.84,15.32,...,131.66,1.30,10.00,-7.00,-64.49,-93.88,-107.84,-8.70,11.15,89.08
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.05,17.45,16.22,...,257.22,4.56,19.13,-11.29,-122.07,-158.45,-207.42,-14.57,15.97,89.17
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,20.18,17.25,15.49,...,233.65,2.48,17.19,-10.41,-87.56,-141.39,-190.84,-14.71,10.26,88.52
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,11.66,11.33,9.96,...,276.05,1.65,20.36,-11.45,-136.27,-187.12,-245.50,-18.71,14.77,90.76


In [12]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36
...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65


In [13]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19,-22.86,-15.33
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00,-7.00,-8.70
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13,-11.29,-14.57
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19,-10.41,-14.71
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36,-11.45,-18.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40,-21.23,-20.52
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18,-16.15,-15.39
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57,-5.59,-11.38
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65,-22.70,-14.41


In [14]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [15]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-,GAA,xGAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19,-22.86,-15.33,-13.441789,-6.156391
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00,-7.00,-8.70,2.418211,0.473609
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13,-11.29,-14.57,-1.871789,-5.396391
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19,-10.41,-14.71,-0.991789,-5.536391
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36,-11.45,-18.71,-2.031789,-9.536391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40,-21.23,-20.52,-11.811789,-11.346391
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18,-16.15,-15.39,-6.731789,-6.216391
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57,-5.59,-11.38,3.828211,-2.206391
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65,-22.70,-14.41,-13.281789,-5.236391


In [16]:
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,GF,GA,G+/-,GAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,2.00,24.86,-22.86,-13.441789
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,2.01,9.01,-7.00,2.418211
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,6.00,17.29,-11.29,-1.871789
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,3.04,13.45,-10.41,-0.991789
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,3.06,14.51,-11.45,-2.031789
...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,0.00,21.23,-21.23,-11.811789
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,0.99,17.14,-16.15,-6.731789
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,0.93,6.52,-5.59,3.828211
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,1.18,23.88,-22.70,-13.281789


In [20]:
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Rates.csv')
print(skater_PP_totals_raw)
skater_PP_totals_raw.head()

                 Player Season Team Position  GP     TOI     GF%    SF%  \
0          Aaron Ekblad  19-20  FLA        D  67   95.90   92.22  81.87   
1          Aaron Ekblad  20-21  FLA        D  35  132.75   91.82  87.33   
2          Aaron Ekblad  21-22  FLA        D  61  214.78   92.51  86.16   
3          Adam Boqvist  19-20  CHI        D  41   87.02   88.20  81.06   
4          Adam Boqvist  20-21  CHI        D  35  118.65   82.03  75.97   
...                 ...    ...  ...      ...  ..     ...     ...    ...   
1303      Zach Werenski  19-20  CBJ        D  63  171.17   80.98  84.22   
1304      Zach Werenski  20-21  CBJ        D  33   73.60  100.00  81.71   
1305      Zach Werenski  21-22  CBJ        D  64  215.48   87.27  79.95   
1306       Zack Kassian  19-20  EDM        R  58   23.18   54.38  91.63   
1307  Zemgus Girgensons  21-22  BUF      C/L  55   54.37  100.00  83.74   

        FF%    CF%  ...  CA/60  xGF/60  xGA/60  G±/60  S±/60  F±/60  C±/60  \
0     84.52  85.63  .

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,81.87,84.52,85.63,...,14.08,6.14,0.45,5.91,39.94,54.16,69.80,5.69,12.57,95.21
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,87.33,87.48,87.89,...,15.16,8.24,0.69,8.92,60.02,79.28,94.85,7.55,13.94,91.44
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,86.16,86.54,87.19,...,14.25,8.21,0.74,9.10,47.97,64.70,82.73,7.47,17.33,91.27
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,81.06,84.35,86.69,...,13.18,6.14,0.63,4.42,36.83,52.28,72.63,5.52,10.62,93.92
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,75.97,79.96,82.55,...,16.41,5.95,1.69,5.32,28.65,42.88,61.26,4.25,16.26,88.74


**Merging Data Frames Help:**
https://github.com/danmartin25/School_District_Analysis/blob/main/PyCitySchools_Challenge.ipynb

In [21]:
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="left", on=["Player", "Player"])
combined_player_df.head(10)

,Player,Season_x,Team_x,Position_x,GP_x,TOI_x,GF%_x,xGF%,GF_x,GA_x,...,xG+/-,Season_y,Team_y,Position_y,GP_y,TOI_y,GF%_y,GF_y,GA_y,GAA
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,...,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,...,1.62,19-20,FLA,D,67.0,95.90,92.22,10.31,0.87,-2.792102
2,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,...,1.62,20-21,FLA,D,35.0,132.75,91.82,21.66,1.93,7.497898
3,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,...,1.62,21-22,FLA,D,61.0,214.78,92.51,35.45,2.87,20.347898
4,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,...,4.73,19-20,FLA,D,67.0,95.90,92.22,10.31,0.87,-2.792102
5,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,...,4.73,20-21,FLA,D,35.0,132.75,91.82,21.66,1.93,7.497898
6,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,...,4.73,21-22,FLA,D,61.0,214.78,92.51,35.45,2.87,20.347898
7,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,...,17.57,19-20,FLA,D,67.0,95.90,92.22,10.31,0.87,-2.792102
8,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,...,17.57,20-21,FLA,D,35.0,132.75,91.82,21.66,1.93,7.497898
9,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,...,17.57,21-22,FLA,D,61.0,214.78,92.51,35.45,2.87,20.347898
